<a href="https://colab.research.google.com/github/anselmo-pitombeira/Notebooks/blob/master/simulacao_carro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install simpy

In [4]:
import simpy
import numpy.random as rd

rd.seed(42)

class Car:

  def __init__(self,env,agente,car_number):
    self.env = env
    self.agente = agente
    self.number = car_number
    self.ride_proc = env.process(self.ride())
    self.start_driving = env.event()    ##O carro espera esse evento ocorrer para iniciar a corrida
    self.busy = False                   ##Variável de estado. True = carro está realizando uma corrida

  def ride(self):

    while True:
        ##Inicia esperando o sinal do agente para começar a corrida
        yield self.start_driving
        print("Car",self.number,": driving at = ",self.env.now)
        self.busy = True
        yield env.timeout(10)    ##Tempo realizando a viagem. Possivelmente criar um processo separado (processo 'ride')
        print("Car", self.number,": Stop driving at = ", self.env.now)
        self.busy = False
        self.agente.free_car.succeed(self.number)    ##Avisa ao agente que está livre
        self.start_driving = env.event()        ##Recria o evento

class Agent:

    def __init__(self,env):
        self.env = env
        self.cars = []                  ##Lista com os carros controlados
        self.lista_calls = []           ##Lista que guarda as calls
        self.free_car = env.event()     ##Esse evento é disparado por um carro quando fica livre
        self.new_call = env.event()     ##Esse evento é disparado quando a lista de calls está vazia e chega uma nova call
        self.controller_proc = env.process(self.control())

    def control(self):

        while True:
            yield self.free_car | self.new_call         ##Espera os eventos free_car ou new_call

            ##Verifica qual evento ocorreu
            if self.new_call.triggered:
                print("Evento nova call no tempo = ",self.env.now)
                self.new_call = env.event()      ##Restabelece o evento

                ##Testa se há algum veículo livre
                veiculo_livre = False
                free_cars = []
                for car in self.cars:
                  if car.busy == False:
                    free_cars.append(car.number)    ##Guarda o número do carro que está livre
                    veiculo_livre = True

                print("Veículos livres = ",free_cars)

                if veiculo_livre == True:   ##Note que se for falso, ignora o evento
                    call1 = self.lista_calls.pop()    ##Retira a call da lista
                    ##Escolhe aleatoriamente um carro da lista de carros livres:
                    car_index = rd.randint(0,len(free_cars))
                    selected_car_number = free_cars[car_index]
                    selected_car = self.cars[selected_car_number]
                    print("Agent: Car", selected_car.number, "start driving at =",self.env.now)
                    selected_car.start_driving.succeed()
                    #selected_car.start_driving = self.env.event()    ##Recria o evento do carro

            if self.free_car.triggered:
                car_number = self.free_car.value  ##Verifica qual carro ficou livre
                print("Carro",car_number,"ficou livre")
                self.free_car = env.event()       ##Restabelece o evento
                if len(self.lista_calls) > 0:
                    print("Lista de calls tem",len(self.lista_calls), "calls")
                    self.lista_calls.pop()       ##Retira uma call da lista. Aqui é que é o cérebro do agente
                    print("Atendendo call")
                    cars[car_number].start_driving.succeed()    ##Manda carro iniciar viagem
                else:
                  print("Lista de chamadas está vazia")

class Call:
  """
  Uma chamada para realizar uma viagem.
  """

  def __init__(self, serial_number, birth_time, dest):
    self.serial_number = serial_number
    self.birth_time = birth_time
    self.dest = dest

def arrival_process(env, tec, agente):

  """
  Processo de chegada de calls, seguindo
  um processo de Poisson.
  """

  serial_numbers = 0
  while True:
    destino = 10
    call = Call(serial_numbers,env.now,destino)
    if len(agente.lista_calls) == 0:
      agente.new_call.succeed()
      #agente.new_call = env.event()   ##Restabelece o evento
    agente.lista_calls.append(call)
    interarrival_time = rd.exponential(tec)
    serial_numbers+=1
    yield env.timeout(interarrival_time)

env = simpy.Environment()
agente = Agent(env)
cars = [Car(env,agente,0), Car(env,agente,1)]
agente.cars = cars    ##Registra os carros no agente
arrival_process_1 = env.process(arrival_process(env,10,agente))
env.run(until=100)



Evento nova call no tempo =  0
Veículos livres =  [0, 1]
Agent: Car 0 start driving at = 0
Car 0 : driving at =  0
Evento nova call no tempo =  4.692680899768591
Veículos livres =  [1]
Agent: Car 1 start driving at = 4.692680899768591
Car 1 : driving at =  4.692680899768591
Evento nova call no tempo =  6.719165942950083
Veículos livres =  []
Car 0 : Stop driving at =  10
Carro 0 ficou livre
Lista de calls tem 1 calls
Atendendo call
Car 0 : driving at =  10
Car 1 : Stop driving at =  14.692680899768591
Carro 1 ficou livre
Lista de chamadas está vazia
Car 0 : Stop driving at =  20
Carro 0 ficou livre
Lista de chamadas está vazia
Evento nova call no tempo =  21.846407681596936
Veículos livres =  [0, 1]
Agent: Car 1 start driving at = 21.846407681596936
Car 1 : driving at =  21.846407681596936
Evento nova call no tempo =  30.93087737791819
Veículos livres =  [0]
Agent: Car 0 start driving at = 30.93087737791819
Car 0 : driving at =  30.93087737791819
Car 1 : Stop driving at =  31.846407681